# ニュース記事のジャンル予測（9分類問題）

事前学習済み日本語T5モデルを、分類タスク用に転移学習（ファインチューニング）します。

- **T5（Text-to-Text Transfer Transformer）**: テキストを入力されるとテキストを出力するという統一的枠組みで様々な自然言語処理タスクを解く深層学習モデル（[日本語解説](https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part7.html)）  
<img src="https://1.bp.blogspot.com/-89OY3FjN0N0/XlQl4PEYGsI/AAAAAAAAFW4/knj8HFuo48cUFlwCHuU5feQ7yxfsewcAwCLcBGAsYHQ/s1600/image2.png">  
出典: [Exploring Transfer Learning with T5: the Text-To-Text Transfer Transformer](https://ai.googleblog.com/2020/02/exploring-transfer-learning-with-t5.html)
- **事前学習**: 個別のタスク用に学習をする前に文法や一般的な言葉の文脈的意味を学習させること（自己教師あり学習とWikipedia等の大規模データ（コーパス）を用いることで広く一般的な知識を持ったモデルを作れる）
- **転移学習、ファインチューニング**: 事前学習済みモデルを初期値にして、特定のタスク用に追加で学習を行うこと（主に教師あり学習）

今回は入出力が次の形式を持ったタスク用に転移学習します。

- **入力**: "{title} {body}"をトークナイズしたトークンID列（最大512トークン）
- **出力**: "{genre_id}"をトークナイズしたトークンID列（最大4トークン）

ここで、{title}はニュース記事のタイトル、{body}は本文、{genre_id}はニュースの分類ラベル（0〜8）です。


# ライブラリやデータの準備

## 依存ライブラリのインストール

In [1]:
!pip install -qU torch==1.7.1 torchtext==0.8.0 torchvision==0.8.2 torchaudio==0.7.2
!pip install -q transformers==4.4.2 pytorch_lightning==1.2.1 sentencepiece
!pip install -q datasets

     |████████████████████████████████| 776.8 MB 16 kB/s 
     |████████████████████████████████| 6.9 MB 22.7 MB/s 
     |████████████████████████████████| 12.8 MB 3.8 MB/s 
     |████████████████████████████████| 7.6 MB 2.2 MB/s 
     |████████████████████████████████| 2.0 MB 5.0 MB/s 
     |████████████████████████████████| 814 kB 22.4 MB/s 
     |████████████████████████████████| 1.3 MB 55.6 MB/s 
     |████████████████████████████████| 880 kB 31.1 MB/s 
     |████████████████████████████████| 3.3 MB 50.9 MB/s 
     |████████████████████████████████| 829 kB 64.3 MB/s 
     |████████████████████████████████| 451 kB 5.2 MB/s 
     |████████████████████████████████| 115 kB 75.2 MB/s 
     |████████████████████████████████| 212 kB 62.2 MB/s 
     |████████████████████████████████| 182 kB 65.0 MB/s 
     |████████████████████████████████| 127 kB 71.0 MB/s 


## 各種ディレクトリ作成

* data: 学習用データセット格納用
* model: 学習済みモデル格納用


In [2]:
!mkdir -p /content/data /content/model

In [3]:
# 事前学習済みモデル
PRETRAINED_MODEL_NAME = "sonoisa/t5-base-japanese"

# 転移学習済みモデルを保存する場所
MODEL_DIR = "/content/model"

## livedoor ニュースコーパスのダウンロード

In [4]:
!wget -O ldcc-20140209.tar.gz https://www.rondhuit.com/download/ldcc-20140209.tar.gz

--2022-12-04 04:32:57--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  2.55MB/s    in 5.0s    

2022-12-04 04:33:03 (1.70 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



## livedoorニュースコーパスの形式変換

livedoorニュースコーパスを次の形式のJSONファイルに変換します。

* "label": ジャンルID（0〜8）
* "text": タイトル + 本文

JSONファイルは/content/dataに格納されます。

## 文字列の正規化の定義

表記揺れを減らします。今回は[neologdの正規化処理](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja)を一部改変したものを利用します。
処理の詳細はリンク先を参照してください。

In [5]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

## 情報抽出

ニュース記事のタイトルと本文とジャンル（9分類）の情報を抽出します。

In [6]:
import tarfile
import re

target_genres = ["dokujo-tsushin",
                 "it-life-hack",
                 "kaden-channel",
                 "livedoor-homme",
                 "movie-enter",
                 "peachy",
                 "smax",
                 "sports-watch",
                 "topic-news"]

def remove_brackets(text):
    text = re.sub(r"(^【[^】]*】)|(【[^】]*】$)", "", text)
    return text

def normalize_text(text):
    assert "\n" not in text and "\r" not in text
    text = text.replace("\t", " ")
    text = text.strip()
    text = normalize_neologd(text)
    text = text.lower()
    return text

def read_title_body(file):
    next(file)
    next(file)
    title = next(file).decode("utf-8").strip()
    title = normalize_text(remove_brackets(title))
    body = normalize_text(" ".join([line.decode("utf-8").strip() for line in file.readlines()]))
    return title, body

genre_files_list = [[] for genre in target_genres]

all_data = []

with tarfile.open("ldcc-20140209.tar.gz") as archive_file:
    for archive_item in archive_file:
        for i, genre in enumerate(target_genres):
            if genre in archive_item.name and archive_item.name.endswith(".txt"):
                genre_files_list[i].append(archive_item.name)

    for i, genre_files in enumerate(genre_files_list):
        for name in genre_files:
            file = archive_file.extractfile(name)
            title, body = read_title_body(file)
            title = normalize_text(title)
            body = normalize_text(body)

            if len(title) > 0 and len(body) > 0:
                all_data.append({
                    "title": title,
                    "body": body,
                    "genre_id": i
                    })

## データ分割

データセットを70% : 15%: 15% の比率でtrain/dev/testに分割します。

* trainデータ: 学習に利用するデータ
* devデータ: 学習中の精度評価等に利用するデータ
* testデータ: 学習結果のモデルの精度評価に利用するデータ

In [7]:
import random
from tqdm import tqdm
import json

random.seed(1234)
random.shuffle(all_data)

def to_line(data):
    title = data["title"]
    body = data["body"]
    genre_id = data["genre_id"]

    assert len(title) > 0 and len(body) > 0
    return json.dumps({
        "label": genre_id,
        "text": f"{title} {body}"
    }, ensure_ascii=False) + "\n"

data_size = len(all_data)
train_ratio, dev_ratio, test_ratio = 0.7, 0.15, 0.15

with open(f"data/train_full.json", "w", encoding="utf-8") as f_train, \
    open(f"data/dev.json", "w", encoding="utf-8") as f_dev, \
    open(f"data/test.json", "w", encoding="utf-8") as f_test:
    
    for i, data in tqdm(enumerate(all_data)):
        line = to_line(data)
        if i < train_ratio * data_size:
            f_train.write(line)
        elif i < (train_ratio + dev_ratio) * data_size:
            f_dev.write(line)
        else:
            f_test.write(line)

7334it [00:00, 49431.74it/s]


作成されたデータを確認します。

形式: { "label": 分類ラベル, "text": 分類する文章 }

In [8]:
!head -3 data/test.json

{"label": 6, "text": "nttドコモ、ジョジョの奇妙な冒険25周年スマホ「jojo l-06d」を発表!荒木飛呂彦氏監修コンテンツが満載の全部入り[optimus_report] オラララオラオラオラオラオラオラオラオラオラオラ!nttドコモは16日、今夏に発売する予定の新モデルや新しく開始するサービスなどを発表する「2012年夏モデル新商品・新サービス発表会」を開催し、人気マンガ「ジョジョの奇妙な冒険」の連載25周年を記念した限定モデル「jojo l-06d」(lgエレクトロニクス製)を発表しています。発売時期は2012年8月を予定しています。jojo l-06dは限定1万5,000台の限定モデルで、5インチサイズの大型ディスプレイを搭載したxi対応androidスマートフォン「optimus vu l-06d」をベースに、原作者・荒木飛呂彦氏が監修したコラボレーションモデルです。荒木氏は監修のほか、jojo l-06dのためだけの書き下ろしイラスト&サインが入っており、ジョジョ好きにはたまらないコンテンツが満載です。コンテンツには、荒木氏が書き下ろした壁紙を含むジョジョの人気イラストの壁紙やライブ壁紙を多数プリインストール。さらに、6種類のきせかえテーマと組み合わせることで、自分だけのお気に入りのホーム画面を設定可能になっています。また、ジョジョ第3部に登場するカーレースゲーム「f-mega」もプリインストールされており、花京院とダービー弟の名勝負を体験できます。さらに、お気に入りのスタンドと合成できるカメラアプリや、トリッシュの電卓、ウェザー・リポートウィジェット、イギーのマチキャラというように作品中に登場するキャラクターによる各種機能、「ジョジョ」の名台詞を織り交ぜたオリジナルの予測変換辞書、オリジナルデコメ絵文字、デコメテンプレートなども搭載。この他、画面サイズが4:3でほぼ文庫サイズのl-06dの端末機能を活かして、特別編集のカラー版コミック第1巻〜12巻も内蔵されています。機能的にも、optimus vu l-06dと同等で、高速データ通信規格lteによるサービス「xi(クロッシィ)」による下り最大75mbpsおよび上り最大25mbpsの高速データ通信や1.5ghzデュアルコアcpu、5インチxga(769×1024ドット)ips液

# 学習に必要なクラス等の定義

学習にはPyTorch/PyTorch-lightning/Transformersを利用します。

In [9]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

## Jsonデータセットクラス

In [10]:
import json

class JsonDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, input_max_len=512, target_max_len=512):
        self.file_path = os.path.join(data_dir, type_path)
        
        self.input_max_len = input_max_len
        self.target_max_len = target_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()
  
    def __len__(self):
        return len(self.inputs)
  
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        source_mask = self.inputs[index]["attention_mask"].squeeze()
        target_mask = self.targets[index]["attention_mask"].squeeze()

        return {"source_ids": source_ids, "source_mask": source_mask, 
                "target_ids": target_ids, "target_mask": target_mask}

    def _make_record(self, text, label):
        # ニュース分類タスク用の入出力形式に変換する。
        input = f"{text}"
        target = f"{label}"
        return input, target
  
    def _build(self):
        with open(self.file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = json.loads(line)
                label = line["label"]
                text = line["text"]

                input, target = self._make_record(text, label)

                tokenized_inputs = self.tokenizer.batch_encode_plus(
                    [input], max_length=self.input_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = self.tokenizer.batch_encode_plus(
                    [target], max_length=self.target_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                self.inputs.append(tokenized_inputs)
                self.targets.append(tokenized_targets)


試しにテストデータ（test.json）を読み込み、トークナイズ結果をみてみます。

In [11]:
# トークナイザー（SentencePiece）モデルの読み込み
tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL_NAME, is_fast=True)

# テストデータセットの読み込み
train_dataset = JsonDataset(tokenizer, "/content/data", "train_full.json", 
                           input_max_len=512, target_max_len=4)

Downloading:   0%|          | 0.00/804k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

テストデータの1レコード目をみてみます。

In [12]:
for data in train_dataset:
    print("A. 入力データの元になる文字列")
    print(tokenizer.decode(data["source_ids"]))
    print()
    print("B. 入力データ（Aの文字列がトークナイズされたトークンID列）")
    print(data["source_ids"])
    print()
    print("C. 出力データの元になる文字列")
    print(tokenizer.decode(data["target_ids"]))
    print()
    print("D. 出力データ（Cの文字列がトークナイズされたトークンID列）")
    print(data["target_ids"])
    break

A. 入力データの元になる文字列
hulu、ついに待望のapple tvに対応!一周年記念で「huluアンバサダープロジェクト」もスタート huluは昨年9月にサービスを開始して以来、日本でのビジネスを順調に拡大してきた。現在は約1000本の映画と、10,000話以上に及ぶテレビ番組の視聴が可能だ。そんなhuluから新たなニュースが飛び込んできた。 ■今日からapple tvに対応!今日、9月4日から新たに「apple tv」にてhuluを楽しめるようになったという。aple idを用いて新規登録や決済も可能だという。apple tvのホーム画面からhuluを起動し、すでにアカウントを持っているユーザーはログイン。また、まだアカウントを持っていない人も簡単にitunesアカウントで登録できる。apple tvより新規登録したユーザーは通常2週間の無料トライアルが1ヶ月になるということだ。 ■サービス一周年記念「huluアンバサダープロジェクト」9月12日からサービス一周年を記念したスペシャルプロジェクト「huluアンバサダープロジェクト」が実施される。facebookやtwitterを通してhuluについて情報発信すると「huluアンバサダー」に任命され、先着10,000名の新規登録者は1ケ月無料トライアルが楽しめるということだ。コンテンツも対応デバイスもますます増えるhuluに今後も注目だ。</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

## 学習処理クラス

[PyTorch-Lightning](https://github.com/PyTorchLightning/pytorch-lightning)を使って学習します。

PyTorch-Lightningとは、機械学習の典型的な処理を簡潔に書くことができるフレームワークです。

In [13]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams

        # 事前学習済みモデルの読み込み
        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)

        # トークナイザーの読み込み
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path, is_fast=True)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, 
                decoder_attention_mask=None, labels=None):
        """順伝搬"""
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

    def _step(self, batch):
        """ロス計算"""
        labels = batch["target_ids"]

        # All labels set to -100 are ignored (masked), 
        # the loss is only computed for labels in [0, ..., config.vocab_size]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            labels=labels
        )

        loss = outputs[0]
        return loss

    def training_step(self, batch, batch_idx):
        """訓練ステップ処理"""
        loss = self._step(batch)
        self.log("train_loss", loss)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        """バリデーションステップ処理"""
        loss = self._step(batch)
        self.log("val_loss", loss)
        return {"val_loss": loss}

    # def validation_epoch_end(self, outputs):
    #     """バリデーション完了処理"""
    #     avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    #     self.log("val_loss", avg_loss, prog_bar=True)

    def test_step(self, batch, batch_idx):
        """テストステップ処理"""
        loss = self._step(batch)
        self.log("test_loss", loss)
        return {"test_loss": loss}

    # def test_epoch_end(self, outputs):
    #     """テスト完了処理"""
    #     avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
    #     self.log("test_loss", avg_loss, prog_bar=True)

    def configure_optimizers(self):
        """オプティマイザーとスケジューラーを作成する"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() 
                            if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() 
                            if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, 
                          lr=self.hparams.learning_rate, 
                          eps=self.hparams.adam_epsilon)

        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=self.hparams.warmup_steps, 
            num_training_steps=self.t_total
        )

        return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]

    def get_dataset(self, tokenizer, type_path, args):
        """データセットを作成する"""
        return JsonDataset(
            tokenizer=tokenizer, 
            data_dir=args.data_dir, 
            type_path=type_path, 
            input_max_len=args.max_input_length,
            target_max_len=args.max_target_length)
    
    def setup(self, stage=None):
        """初期設定（データセットの読み込み）"""
        if stage == 'fit' or stage is None:
            train_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                             type_path="train.json", args=self.hparams)
            self.train_dataset = train_dataset

            val_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                           type_path="dev.json", args=self.hparams)
            self.val_dataset = val_dataset

            self.t_total = (
                (len(train_dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
                // self.hparams.gradient_accumulation_steps
                * float(self.hparams.num_train_epochs)
            )

    def train_dataloader(self):
        """訓練データローダーを作成する"""
        return DataLoader(self.train_dataset, 
                          batch_size=self.hparams.train_batch_size, 
                          drop_last=True, shuffle=True, num_workers=4)

    # def val_dataloader(self):
    #     """バリデーションデータローダーを作成する"""
    #     return DataLoader(self.val_dataset, 
    #                       batch_size=self.hparams.eval_batch_size, 
    #                       num_workers=4)


## few-shot訓練データ作成

各クラスのデータ数がデータ数が8個になるようにする（層化抽出）。

In [14]:
import json
from datasets import load_dataset

class_labels = list(range(9))  # 分類ラベルのリスト
samples_per_class = 64  # クラスあたりのデータ数

train_dataset = load_dataset("json", data_files="/content/data/train_full.json")["train"]
train_dataset = train_dataset.shuffle(seed=5678)

with open("/content/data/train.json", "w", encoding="utf-8") as f_out:
    for class_label in class_labels:
        class_data = train_dataset.filter(lambda x: x["label"] == class_label).select(range(samples_per_class))

        assert len(class_data) == samples_per_class

        for data in class_data:
            f_out.write(json.dumps(data, ensure_ascii=False))
            f_out.write("\n")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-1f936254c4b9ae8c/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

...

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [15]:
# 全データを用いる場合
# !cp /content/data/train_full.json /content/data/train.json 

# 転移学習を実行

In [16]:
# GPU利用有無
USE_GPU = torch.cuda.is_available()

# 各種ハイパーパラメータ
args_dict = dict(
    data_dir="/content/data",  # データセットのディレクトリ
    model_name_or_path=PRETRAINED_MODEL_NAME,
    tokenizer_name_or_path=PRETRAINED_MODEL_NAME,

    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    gradient_accumulation_steps=1,

    n_gpu=1 if USE_GPU else 0,
    early_stop_callback=False,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    seed=42,
)


In [17]:
# 学習に用いるハイパーパラメータを設定する
args_dict.update({
    "max_input_length":  512,  # 入力文の最大トークン数
    "max_target_length": 4,  # 出力文の最大トークン数
    "train_batch_size":  8,
    "eval_batch_size":   8,
    "num_train_epochs":  4,
    })
args = argparse.Namespace(**args_dict)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
)

In [18]:
# 転移学習の実行（GPUを利用すれば1エポック10分程度）
model = T5FineTuner(args)
trainer = pl.Trainer(**train_params)
trainer.fit(model)

# 最終エポックのモデルを保存
model.tokenizer.save_pretrained(MODEL_DIR)
model.model.save_pretrained(MODEL_DIR)

del model

Downloading:   0%|          | 0.00/710 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO:lightning:TPU available: None, using: 0 TPU cores
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)
INFO:lightning:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable param

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


# 学習済みモデルの読み込み

In [19]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

# トークナイザー（SentencePiece）
tokenizer = T5Tokenizer.from_pretrained(MODEL_DIR, is_fast=True)

# 学習済みモデル
trained_model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    trained_model.cuda()

# テストデータに対する予測精度を評価

In [20]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

# テストデータの読み込み
test_dataset = JsonDataset(tokenizer, args_dict["data_dir"], "test.json", 
                          input_max_len=args.max_input_length, 
                          target_max_len=args.max_target_length)

test_loader = DataLoader(test_dataset, batch_size=32, num_workers=4)

trained_model.eval()

outputs = []
confidences = []
targets = []

for batch in tqdm(test_loader):
    input_ids = batch['source_ids']
    input_mask = batch['source_mask']
    if USE_GPU:
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()

    outs = trained_model.generate(input_ids=input_ids, 
        attention_mask=input_mask, 
        max_length=args.max_target_length,
        return_dict_in_generate=True,
        output_scores=True)

    dec = [tokenizer.decode(ids, skip_special_tokens=True, 
                            clean_up_tokenization_spaces=False) 
                for ids in outs.sequences]
    conf = [s.cpu().item() for s in torch.exp(outs.sequences_scores)]
    target = [tokenizer.decode(ids, skip_special_tokens=True, 
                               clean_up_tokenization_spaces=False) 
                for ids in batch["target_ids"]]

    outputs.extend(dec)
    confidences.extend(conf)
    targets.extend(target)
    

  0%|          | 0/35 [00:00<?, ?it/s]

## accuracy

In [21]:
metrics.accuracy_score(targets, outputs)

0.8690909090909091

## ラベル別精度

[accuracy, precision, recall, f1-scoreの意味](http://ibisforest.org/index.php?F値)

In [22]:
print(metrics.classification_report(targets, outputs))

              precision    recall  f1-score   support

           0       0.94      0.75      0.84       130
           1       0.77      0.90      0.83       121
           2       0.87      0.81      0.84       123
           3       0.64      0.79      0.71        82
           4       0.84      0.99      0.91       129
           5       0.91      0.75      0.82       141
           6       0.95      0.96      0.95       127
           7       0.95      0.96      0.95       127
           8       0.95      0.88      0.92       120

    accuracy                           0.87      1100
   macro avg       0.87      0.87      0.86      1100
weighted avg       0.88      0.87      0.87      1100



## 確信度の上下限

In [23]:
min(confidences), max(confidences)

(0.5886873602867126, 0.9999984502792358)